# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [3]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
from skimage.transform import resize
import matplotlib.pyplot as plt
import keras

Using TensorFlow backend.


We set the random seed so that the results don't vary drastically.

In [4]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [4]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
# batch_size = 20 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [5]:
def generator(source_path, folder_list, batch_size, x, y, z):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0, 30,2)]  #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(t)/batch_size) # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image=resize(image,(y,z,3)).astype(np.float32)

                    batch_data[folder,idx,:,:,0] = (image[:,:,0]-image[:,:,0].min())/(image[:,:,0].max()-image[:,:,0].min())#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1]-image[:,:,1].min())/(image[:,:,1].max()-image[:,:,1].min())#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2]-image[:,:,2].min())/(image[:,:,2].max()-image[:,:,2].min())#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(t)%batch_size) != 0:
            batch_data = np.zeros((len(t)%batch_size,x,y,z,3))
            batch_labels = np.zeros((len(t)%batch_size,5))
            for folder in range(len(t)%batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = plt.imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image=resize(image,(y,z,3)).astype(np.float32)
                    batch_data[folder,idx,:,:,0] = (image[:,:,0]-image[:,:,0].min())/(image[:,:,0].max()-image[:,:,0].min()) 
                    batch_data[folder,idx,:,:,1] = (image[:,:,1]-image[:,:,1].min())/(image[:,:,1].max()-image[:,:,1].min()) 
                    batch_data[folder,idx,:,:,2] = (image[:,:,2]-image[:,:,2].min())/(image[:,:,2].max()-image[:,:,2].min()) 

                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1

            yield batch_data, batch_labels


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [6]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)

# training sequences = 663
# validation sequences = 100


In [7]:
def train_model(model, num_epochs, num_image_per_video, image_width, image_height, batch_size):
    # Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.
    
    train_generator = generator(train_path, train_doc, batch_size, num_image_per_video, image_width, image_height)
    val_generator = generator(val_path, val_doc, batch_size, num_image_per_video, image_width, image_height)
    
    model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
    if not os.path.exists(model_name):
        os.mkdir(model_name)

    filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

    LR = ReduceLROnPlateau(monitor='val_loss',factor=0.01,patience=3,min_lr=0.0001,cooldown=1)# write the REducelronplateau code here
    callbacks_list = [checkpoint, LR]
    
    # The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.
    if (num_train_sequences%batch_size) == 0:
        steps_per_epoch = int(num_train_sequences/batch_size)
    else:
        steps_per_epoch = (num_train_sequences//batch_size) + 1

    if (num_val_sequences%batch_size) == 0:
        validation_steps = int(num_val_sequences/batch_size)
    else:
        validation_steps = (num_val_sequences//batch_size) + 1
    
    # Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.
    model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [8]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout, CuDNNLSTM
from keras.layers.convolutional import Conv3D, Conv2D, MaxPooling3D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here

### Model 1 : Conv2D + RNN

In [8]:
class Conv2DModel:
    def define_model(self, num_image_per_video, image_width, image_height):
        model = Sequential()

        model.add(TimeDistributed(Conv2D(8, (3, 3), strides=(2, 2),activation='relu', padding='same'), 
                                    input_shape=(num_image_per_video, image_width, image_height, 3)))
        model.add(TimeDistributed(BatchNormalization()))

        model.add(TimeDistributed(Conv2D(16, (3,3), activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
        model.add(Dropout(0.3))

        model.add(TimeDistributed(Conv2D(64, (2,2),padding='same', activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
        model.add(Dropout(0.3))

        model.add(TimeDistributed(Flatten()))

        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))


        model.add(GRU(128, return_sequences=False))
        model.add(Dense(5, activation='softmax'))
        optimiser = optimizers.Adam()
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
        return model

### Experiment Run 1: Image Res (84 X 84), Epochs = 25, Batch size = 20, Number of images per video = 20

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [9]:
model2D = Conv2DModel()
model2D = model2D.define_model(20,84,84)
print(model2D.summary())

NameError: name 'Sequential' is not defined

In [10]:
train_model(model2D, 25, 20, 84, 84, 20)

Source path =  ./Project_data/val Source path =  ./Project_data/train ; batch size = 20
; batch size = 20Epoch 1/25



/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


34/34 [==============================] - 64s 2s/step - loss: 1.7980 - categorical_accuracy: 0.1957 - val_loss: 1.7854 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to model_init_2020-11-0913_54_06.183482/model-00001-1.80247-0.20060-1.78540-0.23000.h5
Epoch 2/25
34/34 [==============================] - 57s 2s/step - loss: 1.7079 - categorical_accuracy: 0.2134 - val_loss: 1.6537 - val_categorical_accuracy: 0.2100

Epoch 00002: saving model to model_init_2020-11-0913_54_06.183482/model-00002-1.71232-0.20211-1.65371-0.21000.h5
Epoch 3/25
34/34 [==============================] - 57s 2s/step - loss: 1.6152 - categorical_accuracy: 0.2031 - val_loss: 1.6063 - val_categorical_accuracy: 0.2900

Epoch 00003: saving model to model_init_2020-11-0913_54_06.183482/model-00003-1.61598-0.20814-1.60633-0.29000.h5
Epoch 4/25
34/34 [==============================] - 52s 2s/step - loss: 1.6026 - categorical_accuracy: 0.2252 - val_loss: 1.5758 - val_categorical_accuracy: 0.2400

Epoch 00004:

### Experiment Run 2: Image Res (100 X 100), Epochs = 25, Batch size = 20, Number of Images per video = 18

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [15]:
model2D = Conv2DModel()
model2D = model2D.define_model(18,100,100)
print(model2D.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_28 (TimeDis (None, 18, 50, 50, 8)     224       
_________________________________________________________________
time_distributed_29 (TimeDis (None, 18, 50, 50, 8)     32        
_________________________________________________________________
time_distributed_30 (TimeDis (None, 18, 48, 48, 16)    1168      
_________________________________________________________________
time_distributed_31 (TimeDis (None, 18, 48, 48, 16)    64        
_________________________________________________________________
time_distributed_32 (TimeDis (None, 18, 24, 24, 16)    0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 18, 24, 24, 16)    0         
_________________________________________________________________
time_distributed_33 (TimeDis (None, 18, 24, 24, 64)    4160      
__________

In [16]:
train_model(model2D, 25, 18, 100, 100, 20)

Source path =  ./Project_data/val ; batch size = 20
Source path =  ./Project_data/train ; batch size = 20
Epoch 1/25


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


34/34 [==============================] - 60s 2s/step - loss: 1.9150 - categorical_accuracy: 0.1840 - val_loss: 1.7782 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to model_init_2020-11-0913_54_06.183482/model-00001-1.91598-0.18854-1.77817-0.23000.h5
Epoch 2/25
34/34 [==============================] - 52s 2s/step - loss: 1.9249 - categorical_accuracy: 0.1943 - val_loss: 1.7102 - val_categorical_accuracy: 0.2500

Epoch 00002: saving model to model_init_2020-11-0913_54_06.183482/model-00002-1.91747-0.19910-1.71023-0.25000.h5
Epoch 3/25
34/34 [==============================] - 56s 2s/step - loss: 1.9140 - categorical_accuracy: 0.1825 - val_loss: 1.7895 - val_categorical_accuracy: 0.1200

Epoch 00003: saving model to model_init_2020-11-0913_54_06.183482/model-00003-1.90480-0.18703-1.78949-0.12000.h5
Epoch 4/25
34/34 [==============================] - 53s 2s/step - loss: 1.8320 - categorical_accuracy: 0.2259 - val_loss: 1.6464 - val_categorical_accuracy: 0.1800

Epoch 00004:

### Experiment Run 3: Image Res (120 X 120), Epochs = 30, Batch size = 25, Number of Images per video = 30

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [17]:
model2D = Conv2DModel()
model2D = model2D.define_model(30,120,120)
print(model2D.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_37 (TimeDis (None, 30, 60, 60, 8)     224       
_________________________________________________________________
time_distributed_38 (TimeDis (None, 30, 60, 60, 8)     32        
_________________________________________________________________
time_distributed_39 (TimeDis (None, 30, 58, 58, 16)    1168      
_________________________________________________________________
time_distributed_40 (TimeDis (None, 30, 58, 58, 16)    64        
_________________________________________________________________
time_distributed_41 (TimeDis (None, 30, 29, 29, 16)    0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 30, 29, 29, 16)    0         
_________________________________________________________________
time_distributed_42 (TimeDis (None, 30, 29, 29, 64)    4160      
__________

In [18]:
train_model(model2D, 25, 30, 120, 120, 25)

Source path =  ./Project_data/val ; batch size = 25
Source path =  ./Project_data/train ; batch size = 25
Epoch 1/25


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


27/27 [==============================] - 78s 3s/step - loss: 1.7787 - categorical_accuracy: 0.2029 - val_loss: 1.6622 - val_categorical_accuracy: 0.2200

Epoch 00001: saving model to model_init_2020-11-0913_54_06.183482/model-00001-1.77816-0.20513-1.66225-0.22000.h5
Epoch 2/25
27/27 [==============================] - 64s 2s/step - loss: 1.6350 - categorical_accuracy: 0.2248 - val_loss: 1.6100 - val_categorical_accuracy: 0.1800

Epoch 00002: saving model to model_init_2020-11-0913_54_06.183482/model-00002-1.63553-0.22474-1.61001-0.18000.h5
Epoch 3/25
27/27 [==============================] - 69s 3s/step - loss: 1.6130 - categorical_accuracy: 0.2128 - val_loss: 1.6098 - val_categorical_accuracy: 0.1800

Epoch 00003: saving model to model_init_2020-11-0913_54_06.183482/model-00003-1.61324-0.21116-1.60978-0.18000.h5
Epoch 4/25
27/27 [==============================] - 69s 3s/step - loss: 1.6101 - categorical_accuracy: 0.1879 - val_loss: 1.6062 - val_categorical_accuracy: 0.2300

Epoch 00004:

### Model 2 : Convolutional 3D Model without dropout

In [19]:
class Conv3DModel:
    def define_model(self, num_image_per_video, image_width, image_height):
        model=Sequential()
        model.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', 
                           input_shape=(num_image_per_video, image_width, image_height, 3)))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))

        model.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

        model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

        model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(Dense(5, activation='softmax'))
        
        optimiser = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True) #write your optimizer
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
        return model

### Experiment Run 4: Image Res (84 X 84), Epochs = 25, Batch size = 20, Number of Images per video = 20

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [24]:
model3D = Conv3DModel()
model3D = model3D.define_model(20,84,84)
print(model3D.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 20, 84, 84, 64)    5248      
_________________________________________________________________
batch_normalization_23 (Batc (None, 20, 84, 84, 64)    256       
_________________________________________________________________
activation_5 (Activation)    (None, 20, 84, 84, 64)    0         
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 10, 42, 84, 64)    0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 10, 42, 84, 128)   221312    
_________________________________________________________________
batch_normalization_24 (Batc (None, 10, 42, 84, 128)   512       
_________________________________________________________________
activation_6 (Activation)    (None, 10, 42, 84, 128)   0         
__________

In [25]:
train_model(model3D, 25, 20, 84, 84, 20)

Source path =  ./Project_data/val ; batch size = 20
Source path =  ./Project_data/train ; batch size = 20
Epoch 1/25


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


34/34 [==============================] - 74s 2s/step - loss: 2.3677 - categorical_accuracy: 0.3687 - val_loss: 1.8447 - val_categorical_accuracy: 0.4200

Epoch 00001: saving model to model_init_2020-11-0913_54_06.183482/model-00001-2.40286-0.36953-1.84472-0.42000.h5
Epoch 2/25
34/34 [==============================] - 59s 2s/step - loss: 0.8449 - categorical_accuracy: 0.6645 - val_loss: 1.6994 - val_categorical_accuracy: 0.5000

Epoch 00002: saving model to model_init_2020-11-0913_54_06.183482/model-00002-0.84452-0.67270-1.69940-0.50000.h5
Epoch 3/25
34/34 [==============================] - 58s 2s/step - loss: 0.6487 - categorical_accuracy: 0.7564 - val_loss: 1.1411 - val_categorical_accuracy: 0.5700

Epoch 00003: saving model to model_init_2020-11-0913_54_06.183482/model-00003-0.65084-0.75867-1.14113-0.57000.h5
Epoch 4/25
34/34 [==============================] - 60s 2s/step - loss: 0.3966 - categorical_accuracy: 0.8734 - val_loss: 1.3520 - val_categorical_accuracy: 0.4800

Epoch 00004:

### Experiment Run 5: Image Res (100 X 100), Epochs = 25, Batch size = 20, Number of Images per video = 20

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [28]:
model3D = Conv3DModel()
model3D = model3D.define_model(20,100,100)
print(model3D.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_13 (Conv3D)           (None, 20, 100, 100, 64)  5248      
_________________________________________________________________
batch_normalization_31 (Batc (None, 20, 100, 100, 64)  256       
_________________________________________________________________
activation_13 (Activation)   (None, 20, 100, 100, 64)  0         
_________________________________________________________________
max_pooling3d_13 (MaxPooling (None, 10, 50, 100, 64)   0         
_________________________________________________________________
conv3d_14 (Conv3D)           (None, 10, 50, 100, 128)  221312    
_________________________________________________________________
batch_normalization_32 (Batc (None, 10, 50, 100, 128)  512       
_________________________________________________________________
activation_14 (Activation)   (None, 10, 50, 100, 128)  0         
__________

In [29]:
train_model(model3D, 25, 20, 100, 100, 20)

Source path =  ./Project_data/val ; batch size = 20
Source path =  ./Project_data/train ; batch size = 20
Epoch 1/25


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


34/34 [==============================] - 103s 3s/step - loss: 4.0670 - categorical_accuracy: 0.3304 - val_loss: 1.6806 - val_categorical_accuracy: 0.3400

Epoch 00001: saving model to model_init_2020-11-0913_54_06.183482/model-00001-4.13992-0.33032-1.68064-0.34000.h5
Epoch 2/25
34/34 [==============================] - 86s 3s/step - loss: 0.9389 - categorical_accuracy: 0.6019 - val_loss: 1.2889 - val_categorical_accuracy: 0.5500

Epoch 00002: saving model to model_init_2020-11-0913_54_06.183482/model-00002-0.95105-0.60030-1.28893-0.55000.h5
Epoch 3/25
34/34 [==============================] - 86s 3s/step - loss: 0.6359 - categorical_accuracy: 0.7616 - val_loss: 1.0804 - val_categorical_accuracy: 0.5700

Epoch 00003: saving model to model_init_2020-11-0913_54_06.183482/model-00003-0.64763-0.75566-1.08036-0.57000.h5
Epoch 4/25
34/34 [==============================] - 86s 3s/step - loss: 0.4530 - categorical_accuracy: 0.8315 - val_loss: 1.4998 - val_categorical_accuracy: 0.3900

Epoch 00004

KeyboardInterrupt: 

### Model 3 : Convolutional 3D Model with dropout

### Experiment Run 6: Image Res (84 X 84), Epochs = 30, Batch size = 20, Number of images per video = 20

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [6]:
class Conv3DModel_2:
    def define_model(self, num_image_per_video, image_width, image_height):
        model=Sequential()
        model.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', 
                           input_shape=(num_image_per_video, image_width, image_height, 3)))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))

        model.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
        model.add(Dropout(0.25))

        model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dropout(0.5))
        model.add(Dense(512, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(5, activation='softmax'))
        
        optimiser = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True) #write your optimizer
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
        return model

In [7]:
model3D1 = Conv3DModel_2()
model3D1 = model3D1.define_model(20,84,84)
print(model3D1.summary())

NameError: name 'Sequential' is not defined

In [24]:
train_model(model3D1, 30, 20, 84, 84, 20)

Source path = Source path =  Epoch 1/30./Project_data/train ; batch size = 20

 ./Project_data/val ; batch size = 20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


25/34 [=====================>........] - ETA: 19s - loss: 12.1498 - categorical_accuracy: 0.1960

KeyboardInterrupt: 

### Model 4: Convolutional 3D Mode with increased number of layers

In [9]:
class Conv3DModel_1:
    def define_model(self, num_image_per_video, image_width, image_height):
        model=Sequential()
        model.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', 
                           input_shape=(num_image_per_video, image_width, image_height, 3)))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))

        model.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
        model.add(Dropout(0.25))

        model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
        model.add(Dropout(0.25))

        model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

        model.add(Flatten())
        model.add(Dropout(0.5))
        model.add(Dense(512, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(5, activation='softmax'))
        
        optimiser = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True) #write your optimizer
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
        return model

### Experiment Run 7: Image Res (84 X 84), Epochs = 30, Batch size = 20, Number of images per video = 20, Increase number of layers

In [10]:
model3D = Conv3DModel_1()
model3D = model3D.define_model(20,84,84)
print(model3D.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 20, 84, 84, 64)    5248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 20, 84, 84, 64)    256       
_________________________________________________________________
activation_1 (Activation)    (None, 20, 84, 84, 64)    0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 10, 42, 84, 64)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 10, 42, 84, 128)   221312    
_________________________________________________________________
batch_normalization_2 (Batch (None, 10, 42, 84, 128)   512       
_________________________________________________________________
activation_2 (Activation)    (None, 10, 42, 84, 128)   0         
__________

In [11]:
train_model(model3D, 25, 20, 84, 84, 20)

Source path =  ./Project_data/val ; batch size = 20
Source path =  ./Project_data/train ; batch size = 20
Epoch 1/25


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


34/34 [==============================] - 158s 5s/step - loss: 3.3024 - categorical_accuracy: 0.2598 - val_loss: 1.3844 - val_categorical_accuracy: 0.3900

Epoch 00001: saving model to model_init_2020-11-0917_08_40.643412/model-00001-3.34422-0.25792-1.38437-0.39000.h5
Epoch 2/25
34/34 [==============================] - 93s 3s/step - loss: 1.6105 - categorical_accuracy: 0.3127 - val_loss: 1.2549 - val_categorical_accuracy: 0.5300

Epoch 00002: saving model to model_init_2020-11-0917_08_40.643412/model-00002-1.62085-0.31222-1.25487-0.53000.h5
Epoch 3/25
34/34 [==============================] - 67s 2s/step - loss: 1.4299 - categorical_accuracy: 0.4106 - val_loss: 1.2824 - val_categorical_accuracy: 0.4800

Epoch 00003: saving model to model_init_2020-11-0917_08_40.643412/model-00003-1.42702-0.40422-1.28244-0.48000.h5
Epoch 4/25
34/34 [==============================] - 61s 2s/step - loss: 1.3974 - categorical_accuracy: 0.3915 - val_loss: 1.2331 - val_categorical_accuracy: 0.5400

Epoch 00004